13 December 2023

https://github.com/shmuir/phoenix-biodiversity-index

# Biodiversity Intactness Index in Phoenix, AZ

## About this Notebook


## About the Data


### Data Citation


## Data Access

In [1]:
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt
import pandas as pd

from shapely.geometry import Polygon
import matplotlib.patches as mpatches # for creating legends


# used to access STAC catalogs
from pystac_client import Client
#used to sign items from the MPC STAC catalog
import planetary_computer
# other libraries for nice outputs
from IPython.display import Image
import contextily as cx

In [2]:
phoenix_census = gpd.read_file('tl_2022_04_cousub/tl_2022_04_cousub.shp')

In [ ]:
# access catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

bii_collection = catalog.get_child('io-biodiversity')
bii_collection

# phoenix bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            
            [-111.17544349576511, 33.867435662315955],
            [-112.82552110133373, 33.867435662315955],
            [-112.82552110133373, 32.97048235813784],
            [-111.17544349576511, 32.97048235813784],
            [-111.17544349576511, 33.867435662315955]
            
        ]
    ],
}

# catalog search
search = catalog.search(
    collections=['io-biodiversity'],
    intersects=bbox)

items = search.item_collection()
items

In [ ]:
bii2017 = items[3]
bii2020 = items[0]

In [ ]:
#Image(url=bii2017.assets['rendered_preview'].href, width=500)

In [ ]:
bii2017.assets

In [ ]:
phoenix2017 = rioxr.open_rasterio(bii2017.assets['data'].href)
phoenix2020 = rioxr.open_rasterio(bii2020.assets['data'].href)

phoenix2017

In [ ]:
phoenix2017 = phoenix2017.squeeze()
phoenix2017 = phoenix2017.drop('band')

phoenix2020 = phoenix2020.squeeze()
phoenix2020 = phoenix2020.drop('band')

In [ ]:
#phoenix.plot()

In [ ]:
phoenix_census.NAME.unique

In [ ]:
phoenix_geom = phoenix_census[phoenix_census.NAME == 'Phoenix']

In [ ]:
ax = phoenix_geom.plot(facecolor="none",
                   edgecolor="red",
                   linewidth=2
                  )
cx.add_basemap(ax,
               crs=phoenix_census.crs.to_string(),
               source=cx.providers.CartoDB.Voyager
              )

## Percent 

In [ ]:
phoenix2017_filter = (phoenix2017 >= 0.75).astype('int')
phoenix2020_filter = (phoenix2020 >= 0.75).astype('int')

In [ ]:
pix_counts_2017 = {'code' : np.unique(phoenix2017_filter, return_counts = True)[0],
     'num_pix' : np.unique(phoenix2017_filter, return_counts = True)[1],
     }

# create data frame
pix_counts_2017 = pd.DataFrame(pix_counts_2017)
pix_counts_2017 = pix_counts_2017[pix_counts_2017.code == 1]
pix_counts_2017

In [ ]:
pix_counts_2020 = {'code' : np.unique(phoenix2020_filter, return_counts = True)[0],
     'num_pix' : np.unique(phoenix2020_filter, return_counts = True)[1],
     }

# create data frame
pix_counts_2020 = pd.DataFrame(pix_counts_2020)
pix_counts_2020 = pix_counts_2020[pix_counts_2020.code == 1]
pix_counts_2020

In [ ]:
(pix_counts_2017.num_pix / (phoenix2017.size)) * 100

In [ ]:
(pix_counts_2020.num_pix / (phoenix2020.size)) * 100

## Mapping

In [ ]:
lost_area = (phoenix2017_filter - phoenix2020_filter)
#lost_area.where(lost_area == 1).plot()

In [16]:
lost_area = lost_area.where(lost_area == 1)  

array([ 1., nan])

In [ ]:
fig, ax = plt.subplots(figsize=(12,8)) # initialize plot

#ax.axis("off") # remove axis

phoenix2020.plot(ax=ax) # plot lost area

lost_area.plot(ax=ax, color='red')
area_patch = mpatches.Patch(color='red', # change color to red
                          label='Area with BII >= 0.75 lost from 2017 to 2020') # update label


ax.legend(handles = [area_patch], frameon=False, loc = (0.75, 0.95)) # add legend

plt.show()